# Clustering Crypto

In [2]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [4]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv("crypto_data.csv", index_col=0)

In [5]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"] == True]

In [6]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=["IsTrading"])

In [7]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [8]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]

In [9]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)

In [10]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=["CoinName"])

In [11]:
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [15]:
# Use get_dummies() to create variables for text features.
X_unscaled = pd.get_dummies(crypto_df)

In [16]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X_unscaled)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [22]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

In [58]:
# Create a DataFrame with the three principal components.
pcs = pca.fit_transform(X)

pcs_df = pd.DataFrame(data=pcs, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

pcs_df

,PC 1,PC 2,PC 3
42,-0.151159,-0.067395,-0.049207
404,-0.133810,-0.066816,-0.052671
1337,0.356948,0.015360,-0.004890
BTC,-0.270229,-0.146783,-0.065297
ETH,-0.167332,-0.175458,-0.155394
...,...,...,...
ZEPH,2.682024,-0.135410,-0.160790
GAP,-0.194119,-0.096789,-0.075258
BDX,-0.254658,-0.273125,-0.183003
ZEN,-0.347215,-0.188088,-0.217276


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [59]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values.
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# Create the Elbow Curve using hvPlot.
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

c:\Users\mitch\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [62]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

In [63]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1, sort=False)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = crypto_names_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.151159,-0.067395,-0.049207,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.133810,-0.066816,-0.052671,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.356948,0.015360,-0.004890,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.270229,-0.146783,-0.065297,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.167332,-0.175458,-0.155394,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.325873,-0.184999,-0.158120,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.078672,-0.022702,0.020308,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.163941,-0.229901,-0.142288,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.368058,-0.156668,-0.164000,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.347214,-0.188088,-0.217276,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [65]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"],
)

fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [66]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [67]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [70]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])

In [71]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=scaler, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [72]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)